In [1]:
from openfermion import (
    QubitOperator as Q,
    commutator,
    anticommutator,
    get_sparse_operator as gso,
    get_ground_state as ggs,
    variance,
    bravyi_kitaev,
    FermionOperator,
    normal_ordered,
    get_majorana_operator
)
import numpy as np
from fermi_util import get_commutator
import pickle
import random
from scipy.optimize import minimize

In [2]:
def total_num_op(num_modes):
    """
    Calculates the total number operator for a given number of modes.
    :param num_modes:
    :return:
    """
    total_number_operator = FermionOperator()
    for mode in range(num_modes):
        total_number_operator += FermionOperator(((mode, 1), (mode, 0)))

    return total_number_operator

In [3]:
def callback(xk, trunc_H, num_modes, num_elec,):
    print(f"Current 1-Norm: {bliss_cost_fn(xk, trunc_H, num_modes, num_elec,)}")

In [4]:
def construct_symmetric_fermion_operator(sym_matrix):
    """
    Constructs a symmetric FermionOperator from a symmetric matrix.

    Args:
        sym_matrix (ndarray): An n x n symmetric matrix.

    Returns:
        FermionOperator: The constructed symmetric FermionOperator.
    """
    n = sym_matrix.shape[0]

    # Check if the matrix is symmetric
    if not np.allclose(sym_matrix, sym_matrix.T):
        raise ValueError("The input matrix must be symmetric.")

    fermion_operator = FermionOperator()

    # Iterate over the matrix elements to construct the operator
    for i in range(n):
        for j in range(i, n):  # Only loop over the upper triangle (i <= j)
            if sym_matrix[i][j] != 0:
                coefficient = sym_matrix[i][j]
                if i == j:
                    # Diagonal terms (e.g., c_i^† c_i)
                    fermion_operator += FermionOperator(((i, 1), (i, 0)),
                                                        coefficient)
                else:
                    # Off-diagonal terms contribute twice (symmetric part)
                    fermion_operator += FermionOperator(((i, 1), (j, 0)),
                                                        coefficient)
                    fermion_operator += FermionOperator(((j, 1), (i, 0)),
                                                        coefficient)

    return fermion_operator

In [5]:
def blissed_H(params, trun_H, num_modes, num_elec):
    total_number_operator = total_num_op(num_modes)
    mu1 = params[0]
    mu2 = params[1]
    mu3 = params[2]
    # matrix = params[3:].reshape((num_modes, num_modes))
    upper_tri_indices = np.triu_indices(num_modes)
    sym_matrix = np.zeros((num_modes, num_modes))
    sym_matrix[upper_tri_indices] = params[3:3 + int(
        num_modes * (num_modes + 1) // 2)]
    sym_matrix = sym_matrix + sym_matrix.T - np.diag(np.diag(sym_matrix))

    upper_tri_indices2 = np.triu_indices(num_modes)
    sym_matrix2 = np.zeros((num_modes, num_modes))
    sym_matrix2[upper_tri_indices2] = params[3 + int(
        num_modes * (num_modes + 1) // 2):]
    sym_matrix2 = sym_matrix2 + sym_matrix2.T - np.diag(np.diag(sym_matrix2))

    truncated_H = FermionOperator()
    truncated_H += trun_H

    truncated_H -= mu1 * (total_number_operator - num_elec)
    truncated_H -= mu2 * (
                total_number_operator * total_number_operator - num_elec ** 2)
    truncated_H -= mu3 * (
                total_number_operator * total_number_operator * total_number_operator - num_elec ** 3)
    O = construct_symmetric_fermion_operator(sym_matrix)
    O_2 = construct_symmetric_fermion_operator(sym_matrix2)
    # O = construct_non_symmetric_fermion_operator(matrix)
    truncated_H -= normal_ordered(O) * (total_number_operator - num_elec)
    truncated_H -= normal_ordered(O_2) * (
                total_number_operator * total_number_operator - num_elec ** 2)

    return truncated_H

In [6]:
def bliss_three_body(trunc_H: FermionOperator, num_modes, num_elec):

    initial_mu = [0, 0, 0]
    init_O = [random.random() for _ in range(int(num_modes * (num_modes + 1)))]
    # init_O = [random.random() for _ in range(num_modes ** 2)]
    initial_guess = np.concatenate((initial_mu, init_O))
    res = minimize(bliss_cost_fn, initial_guess, args=(trunc_H, num_modes, num_elec,), method='BFGS',
                   options={'gtol': 1e-30, 'disp': True, 'maxiter': 100}, callback=lambda xk: callback(xk, trunc_H))

    bliss_result = blissed_H(res.x, trunc_H, num_modes, num_elec)

    return bliss_result

In [7]:
def bliss_cost_fn(params, trun_H, num_modes, num_elec):
    total_number_operator = total_num_op(num_modes)
    mu1 = params[0]
    mu2 = params[1]
    mu3 = params[2]
    # matrix = params[3:].reshape((num_modes, num_modes))
    upper_tri_indices = np.triu_indices(num_modes)
    sym_matrix = np.zeros((num_modes, num_modes))
    sym_matrix[upper_tri_indices] = params[3:3 + int(
        num_modes * (num_modes + 1) // 2)]
    sym_matrix = sym_matrix + sym_matrix.T - np.diag(np.diag(sym_matrix))

    upper_tri_indices2 = np.triu_indices(num_modes)
    sym_matrix2 = np.zeros((num_modes, num_modes))
    sym_matrix2[upper_tri_indices2] = params[3 + int(
        num_modes * (num_modes + 1) // 2):]
    sym_matrix2 = sym_matrix2 + sym_matrix2.T - np.diag(
        np.diag(sym_matrix2))
    truncated_H = FermionOperator()
    truncated_H += trun_H

    truncated_H -= mu1 * (total_number_operator - num_elec)
    truncated_H -= mu2 * (
                total_number_operator * total_number_operator - num_elec ** 2)
    truncated_H -= mu3 * (
                total_number_operator * total_number_operator * total_number_operator - num_elec ** 3)
    O = construct_symmetric_fermion_operator(sym_matrix)
    O_2 = construct_symmetric_fermion_operator(sym_matrix2)
    # O = construct_non_symmetric_fermion_operator(matrix)
    truncated_H -= normal_ordered(O) * (total_number_operator - num_elec)
    truncated_H -= normal_ordered(O_2) * (
                total_number_operator * total_number_operator - num_elec ** 2)
    majo = get_majorana_operator(truncated_H)

    one_norm = 0
    for term, coeff in majo.terms.items():
        one_norm += abs(coeff)
    return one_norm


In [13]:
moltag = 'h2'

num_modes = 4
num_elec = 2

filename = f'ham_lib/{moltag}_fer.bin'
with open(filename, 'rb') as f:
    H = pickle.load(f)
print(H)
#G = FermionOperator('0^ 1^ 3 2') - FermionOperator('2^ 3^ 1 0')
# G = (FermionOperator('0^ 1^ 0 1') - FermionOperator('1^ 0^ 1 0') 
#      + FermionOperator('1^ 2^ 1 2') - FermionOperator('2^ 1^ 2 1') 
#      + FermionOperator('0^ 3^ 0 3') - FermionOperator('3^ 0^ 3 0') 
#      + FermionOperator('1^ 3^ 1 3') - FermionOperator('3^ 1^ 3 1'))

G = FermionOperator('0^ 2') - FermionOperator('2^ 0') + FermionOperator('1^ 3') - FermionOperator('3^ 1')
    

commutator = get_commutator(H, G)
print(commutator)
print("Commutator Obtained in Fermion and Qubit space")
with open(f'commutator_store/{moltag}_commutator.pkl', 'wb') as f:
    pickle.dump(commutator, f)

0.52917721092 [] +
-1.110844179883727 [0^ 0] +
0.31320124976475894 [0^ 0^ 0 0] +
0.09839529174273512 [0^ 0^ 2 2] +
0.31320124976475894 [0^ 1^ 1 0] +
0.09839529174273512 [0^ 1^ 3 2] +
0.09839529174273512 [0^ 2^ 0 2] +
0.3108533815598566 [0^ 2^ 2 0] +
0.09839529174273512 [0^ 3^ 1 2] +
0.3108533815598566 [0^ 3^ 3 0] +
0.31320124976475894 [1^ 0^ 0 1] +
0.09839529174273512 [1^ 0^ 2 3] +
-1.110844179883727 [1^ 1] +
0.31320124976475894 [1^ 1^ 1 1] +
0.09839529174273512 [1^ 1^ 3 3] +
0.09839529174273512 [1^ 2^ 0 3] +
0.3108533815598566 [1^ 2^ 2 1] +
0.09839529174273512 [1^ 3^ 1 3] +
0.3108533815598566 [1^ 3^ 3 1] +
0.31085338155985665 [2^ 0^ 0 2] +
0.09839529174273512 [2^ 0^ 2 0] +
0.31085338155985665 [2^ 1^ 1 2] +
0.09839529174273512 [2^ 1^ 3 0] +
-0.5891210037060828 [2^ 2] +
0.09839529174273512 [2^ 2^ 0 0] +
0.3265353734712869 [2^ 2^ 2 2] +
0.09839529174273512 [2^ 3^ 1 0] +
0.3265353734712869 [2^ 3^ 3 2] +
0.31085338155985665 [3^ 0^ 0 3] +
0.09839529174273512 [3^ 0^ 2 1] +
0.3108533815598566

In [14]:
majo = get_majorana_operator(commutator)
    
one_norm = 0
for term, coeff in majo.terms.items():
    one_norm += abs(coeff)
print(one_norm)

1.82121721347756


In [15]:
# import h5py
# from utils.physicist_to_chemist import physicist_to_chemist, \
#     chemist_ferm_to_tensor
# 
# commutator_phy = physicist_to_chemist(commutator)
# obt, tbt, threebt = chemist_ferm_to_tensor(commutator_phy, num_modes // 2)
# print(obt, tbt, threebt)
# # Open or create an HDF5 file
# with h5py.File(f"{moltag}_commutator_tensors.h5", "w") as fid:
# 
#     # Create another group named "BLISS_HAM"
#     mol_data_group = fid.create_group("BLISS_HAM")
# 
#     mol_data_group["h_const"] = 0
#     mol_data_group["obt"] = obt
#     mol_data_group["tbt"] = tbt
#     mol_data_group["threebt"] = threebt  # Assuming this is intentional duplication
#     mol_data_group["eta"] = num_elec  # Replace with actual num_electrons value

In [16]:
print(commutator)

-0.5217231761776442 [0^ 2] +
-0.3888854305611358 [1^ 0^ 2 1] +
0.3888854305611357 [1^ 0^ 3 0] +
-0.5217231761776442 [1^ 3] +
-0.5217231761776442 [2^ 0] +
-0.3888854305611358 [2^ 1^ 1 0] +
0.3622171831480798 [2^ 1^ 3 2] +
0.3888854305611358 [3^ 0^ 1 0] +
-0.3622171831480798 [3^ 0^ 3 2] +
-0.5217231761776442 [3^ 1] +
0.3622171831480798 [3^ 2^ 2 1] +
-0.3622171831480798 [3^ 2^ 3 0]


In [17]:
initial_mu = [0, 0, 0]
# init_O = [random.random() for _ in range(int(num_modes * (num_modes + 1)//2))]
init_O = [random.random() for _ in range(int(num_modes * (num_modes + 1)))]
# init_O = [random.random() for _ in range(num_modes ** 2)]
initial_guess = np.concatenate((initial_mu, init_O))
res = minimize(bliss_cost_fn, initial_guess, args=(commutator, num_modes, num_elec,), method='BFGS',
               options={'gtol': 1e-30, 'disp': True, 'maxiter': 30}, callback=lambda xk: callback(xk, commutator, num_modes, num_elec,))


Current 1-Norm: 19.60517843413562
Current 1-Norm: 15.695913645122515


KeyboardInterrupt: 

In [25]:
# loaded_array = np.load('H2O_Bliss_params.npy')
# print(loaded_array)

In [15]:
res = minimize(bliss_cost_fn, res.x, args=(commutator, num_modes, num_elec,), method='BFGS',
               options={'gtol': 1e-30, 'disp': True, 'maxiter': 30}, callback=lambda xk: callback(xk, commutator, num_modes, num_elec,))

Current 1-Norm: 1.4637247999247538
Current 1-Norm: 1.4637239716736243
Current 1-Norm: 1.4637230486164357
Current 1-Norm: 1.4637228571782959
Current 1-Norm: 1.4637227202861416
Current 1-Norm: 1.4637225531626201
Current 1-Norm: 1.4637222878129816
Current 1-Norm: 1.4637218540768837
Current 1-Norm: 1.4637211472210785
Current 1-Norm: 1.4637210807340042
Current 1-Norm: 1.4637210612462968
Current 1-Norm: 1.4637210449582199
Current 1-Norm: 1.4637210095876425
Current 1-Norm: 1.463720945174511
Current 1-Norm: 1.4637208288631942
Current 1-Norm: 1.4637198169179872
Current 1-Norm: 1.463717893056038
Current 1-Norm: 1.4637161514552184
Current 1-Norm: 1.4637157041250886
Current 1-Norm: 1.4637134271998715
         Current function value: 1.463713
         Iterations: 20
         Function evaluations: 2436
         Gradient evaluations: 101


/Users/rick/PycharmProjects/croor/.venv/lib/python3.9/site-packages/scipy/optimize/_minimize.py:708: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [36]:
np.save('H2O_Bliss_params.npy', res.x)

In [16]:
# bliss_output = bliss_three_body(commutator, N, N//2)
bliss_output = blissed_H(res.x, commutator, num_modes, num_elec)
print("BLISS Complete. Obtained in Fermion and Qubit space")
with open(f'commutator_store/{moltag}_commutator_blissed.pkl', 'wb') as f:
    pickle.dump(bliss_output, f)

BLISS Complete. Obtained in Fermion and Qubit space


In [19]:
import h5py
from utils.physicist_to_chemist import physicist_to_chemist, chemist_ferm_to_tensor

In [20]:
commutator_phy = physicist_to_chemist(commutator)
obt, tbt, threebt = chemist_ferm_to_tensor(commutator_phy, num_modes//2)

In [21]:
# Open or create an HDF5 file
with h5py.File(f"{moltag}_commutator_tensors.h5", "w") as fid:

    # Create another group named "BLISS_HAM"
    mol_data_group = fid.create_group("BLISS_HAM")

    mol_data_group["h_const"] = 0
    mol_data_group["obt"] = obt
    mol_data_group["tbt"] = tbt
    mol_data_group["threebt"] = threebt  # Assuming this is intentional duplication
    mol_data_group["eta"] = num_elec  # Replace with actual num_electrons value

In [15]:
from openfermion import *
ferm_op = FermionOperator("0^ 1^ 3 2") + FermionOperator('2^ 3^ 1 0')
Hqub = jordan_wigner(ferm_op)

In [16]:
for term in Hqub.terms:
    print(Hqub.terms[term])

(0.125+0j)
(0.125+0j)
(-0.125+0j)
(0.125+0j)
(0.125+0j)
(-0.125+0j)
(0.125+0j)
(0.125+0j)


In [17]:
sorted_terms = sorted(Hqub.terms, key=lambda x: np.abs(Hqub.terms[x]), reverse=True)

In [18]:
print(sorted_terms)

[((0, 'Y'), (1, 'X'), (2, 'Y'), (3, 'X')), ((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')), ((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X')), ((0, 'Y'), (1, 'Y'), (2, 'Y'), (3, 'Y')), ((0, 'X'), (1, 'X'), (2, 'X'), (3, 'X')), ((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y')), ((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X')), ((0, 'X'), (1, 'Y'), (2, 'X'), (3, 'Y'))]


In [19]:
def qubit_wise_commute(this, other):
    qubit_term = {} #Dictionary of qubit terms in self.
    for qub in range(this.n_qubit):
        cur_qub_term = (this.binary[qub], this.binary[qub + this.n_qubit]) 
        if cur_qub_term != (0, 0):
            qubit_term[qub] = cur_qub_term
    
    for qub in range(other.n_qubit):
        cur_qub_term = (other.binary[qub], other.binary[qub + this.n_qubit]) 
        if cur_qub_term != (0, 0):
            if qub in qubit_term:
                if qubit_term[qub] != cur_qub_term: return False
    return True

In [20]:
def term_commutes_with_group(term, group):
    '''
    Returns if term commutes with a group of terms.
    '''
    commute = True
    for group_term in group:
        commute = qubit_wise_commute(term, group_term)
        if not (commute): break
    return commute

In [26]:
from paulis import PauliString
groups = []
for term in sorted_terms:
    pauli_str = PauliString(term)
    print(pauli_str)
    found_group = False
    for idx, group in enumerate(groups):
        commute = True
        for paulistring in group:
            group_pauli = PauliString(paulistring)
            print(PauliString)
            commute = pauli_str.qubit_wise_commute(group_pauli)
        if commute:
            groups[idx].append(term)
            ground_group = True
            break
    if not found_group: groups.append([term,])
print(groups)

{0: 'Y', 1: 'X', 2: 'Y', 3: 'X'}
{0: 'Y', 1: 'X', 2: 'X', 3: 'Y'}
<class 'paulis.PauliString'>
{0: 'Y', 1: 'Y', 2: 'X', 3: 'X'}
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
{0: 'Y', 1: 'Y', 2: 'Y', 3: 'Y'}
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
{0: 'X', 1: 'X', 2: 'X', 3: 'X'}
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
{0: 'X', 1: 'X', 2: 'Y', 3: 'Y'}
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
{0: 'X', 1: 'Y', 2: 'Y', 3: 'X'}
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
{0: 'X', 1: 'Y', 2: 'X', 3: 'Y'}
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'paulis.PauliString'>
<class 'pau

In [27]:
decomp = []
for group in groups:
    qubit_op = QubitOperator()
    for paulistring in group:
        qubit_op += PauliString(paulistring).to_qubit_operator()
    
    decomp.append(qubit_op)

In [28]:
print(decomp)

[1.0 [Y0 X1 Y2 X3], 1.0 [Y0 X1 X2 Y3], 1.0 [Y0 Y1 X2 X3], 1.0 [Y0 Y1 Y2 Y3], 1.0 [X0 X1 X2 X3], 1.0 [X0 X1 Y2 Y3], 1.0 [X0 Y1 Y2 X3], 1.0 [X0 Y1 X2 Y3]]


In [24]:
for i in groups:
    print(i)

[((0, 'Y'), (1, 'X'), (2, 'Y'), (3, 'X'))]
[((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y'))]
[((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X'))]
[((0, 'Y'), (1, 'Y'), (2, 'Y'), (3, 'Y'))]
[((0, 'X'), (1, 'X'), (2, 'X'), (3, 'X'))]
[((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y'))]
[((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X'))]
[((0, 'X'), (1, 'Y'), (2, 'X'), (3, 'Y'))]
